# Layers and Blocks

To begin, we revisit the code
that we used to implement MLPs

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.0275,  0.0889, -0.1878, -0.0165,  0.0356, -0.3514, -0.0725,  0.0725,
          0.0508, -0.0560],
        [ 0.0224,  0.0984, -0.2499,  0.0313,  0.1201, -0.5500, -0.1515,  0.1012,
          0.0086, -0.0528]], grad_fn=<AddmmBackward0>)

`nn.Sequential` defines a special kind of `Module`

A Custom Block

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

Instantiate the MLP's layers
and subsequently invoke these layers

In [3]:
net = MLP()
net(X)

tensor([[ 0.1298,  0.2597, -0.0968, -0.0494,  0.1476,  0.1454,  0.1632,  0.0199,
         -0.0087,  0.3480],
        [ 0.1466,  0.1623, -0.0858, -0.0544,  0.1312,  0.2089,  0.1057,  0.0484,
         -0.0889,  0.3298]], grad_fn=<AddmmBackward0>)

The Sequential Block

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.2045, -0.1206,  0.2077,  0.0615, -0.1246,  0.0457, -0.0679, -0.3117,
          0.0269, -0.3475],
        [-0.2765, -0.2260,  0.1733,  0.1343, -0.3023,  0.0324, -0.1404, -0.4459,
         -0.0769, -0.2924]], grad_fn=<AddmmBackward0>)

Executing Code in the Forward Propagation Function

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.1288, grad_fn=<SumBackward0>)

Mix and match various
ways of assembling blocks together

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.2879, grad_fn=<SumBackward0>)